## `Non-Repetitive Parts Calculator` in Action!

**Authors** Ayaan Hossain and Howard Salis

**Updated** August 2, 2020

Click on the title **`Non-Repetitive` ... Action!** above and press `Shift`+`Enter` together on each cell to follow along and see all the code in action. If you're previewing this on GitHub, please [download](https://github.com/ayaanhossain/nrpcalc#NRP-Calculator-in-Action) the `nrpcalc` repository to execute this notebook locally. TOC links may not work in GitHub preview.

This `jupyter` notebook will demonstrate the usage of the `Non-Repetitive Parts Calculator` and describe how to use its [API](https://github.com/ayaanhossain/nrpcalc/blob/master/docs/DOCS.md) to design commonly used genetic parts. Our purpose here is to demonstrate the different features of the `Non-Repetitive Parts Calculator` and to illustrate the considerations involved when designing thousands of non-repetitive genetic parts. Of course, the `Non-Repetitive Parts Calculator` may be used to design many different types of genetic parts beyond the examples presented here. The design possibilities are quite open-ended with this algorithm.

### Table of Contents

* [Notebook Setup](#Notebook-Setup)

* [Constraint Based Design of Genetic Parts](#Constraint-Based-Design-of-Genetic-Parts)

* [Non-Repetitive σ<sup>70</sup> Promoters with CRISPRi with `Lmax=12`](#Non-Repetitive-σ70-Promoters-with-CRISPRi-with-Lmax=12)
    * [Designing the First Toolbox](#Designing-the-First-Toolbox)
    * [Designing the Second Toolbox - First Attempt](#Designing-the-Second-Toolbox---First-Attempt)
    * [Designing the Second Toolbox - Second Attempt](#Designing-the-Second-Toolbox---Second-Attempt)

* [Non-Repetititve Ribosome Binding Sites with `Lmax=14`](#Non-Repetititve-Ribosome-Binding-Sites-with-Lmax=14)

* [Non-Repetitive Toehold Switches with `Lmax=14`](#Non-Repetitive-Toehold-Switches-with-Lmax=14)

* [Non-Repetitive Intrinsic Terminators with `Lmax=14`](#Non-Repetitive-Intrinsic-Terminators-with-Lmax=14)

* [And Now, Our Watch is Ended](#And-Now,-Our-Watch-is-Ended)

* [References](#References)

### Notebook Setup

If you [installed](https://github.com/ayaanhossain/nrpcalc#Installation) `nrpcalc` successfully, you have everything you need to follow along.

Let's first import `nrpcalc`.

In [1]:
import nrpcalc

Hopefully, the import worked without throwing up any errors! If you face issues importing, please [open an issue](https://github.com/ayaanhossain/nrpcalc/issues). If everything worked fine, you're ready to follow along. If you do not understand a specific part of this notebook, either open an issue, or please reach the authors via Twitter or Email. We would be happy to answer your questions, and update this notebook in response to your questions, comments or concerns.

In [2]:
print(nrpcalc.__doc__) # show docs


Non-Repetitive Parts Calculator

Automated design and discovery of non-repetitive genetic
parts for engineering stable systems.

Version: 1.3.2

Authors: Ayaan Hossain <auh57@psu.edu>
         Howard Salis  <salis@psu.edu>

The Non-Repetitive Parts Calculator offers two modes of operation:

- Finder Mode: Discover toolboxes of non-repetitive parts
               from a list of candidate parts

-  Maker Mode: Design toolboxes of non-repetitive parts
               based on sequence, structure and model
               constraints

Additionally, a 'background' object is available that stores
background sequences. When the 'background' object is used, 
designed genetic parts will also be non-repetitive with respect
to these sequences.

You can learn more about the two modes and background via
  print(nrpcalc.background.__doc__)
  print(nrpcalc.finder.__doc__)
  print(nrpcalc.maker.__doc__)



In [3]:
import time # time-keeping is important!

### Constraint Based Design of Genetic Parts

Genetic parts exhibit their activity through biophysical interactions that rely on **DNA** or **RNA** sequence motifs, the presence or absence of specific RNA structures, and/or higher-order sequence or structural characteristics. For example, a constitutive σ<sup>70</sup> _E. coli_ promoter sequence will have a high transcription initiation rate when it contains a conserved $-35$ and $-10$ hexamer, separated by a $17$ base pair spacer. Likewise, a bacterial transcriptional terminator will have a high efficiency when it contains a fast-folding, stable RNA hairpin, followed by a U-rich tract.

Such essential characteristics can be flexibly distilled into a set of criteria that every generated genetic part sequence must satisfy. The `Non-Repetitive Parts Calculator` `Maker Mode` accepts three types of genetic part constraints: a degenerate DNA or RNA sequence using the **IUPAC code**; an essential RNA secondary structure using _dot-parenthesis-x_ notation; and a model-based constraint that can be customized to quantify the presence of higher-order interactions or to facilitate the synthesis and construction of the genetic system by excluding sequences (e.g. restriction sites and polymeric sequences). All three constraints may be simultaneously used to specify a set of genetic part sequences with desired functionalities.

As examples, **Supplementary Table 2** of our [publication](https://static-content.springer.com/esm/art%3A10.1038%2Fs41587-020-0584-2/MediaObjects/41587_2020_584_MOESM1_ESM.pdf) (see page 30) lists the design constraints and algorithm outcomes for a wide variety of genetic parts commonly used in synthetic biology, including minimal Pol II promoters, insulated ribosome binding sites, prokaryotic transcriptional terminators, and toehold RNA switches.

In one sense, these genetic part constraints are explicit hypotheses that distill one’s knowledge of gene regulation and biophysics into the simplest possible computable form. In another sense, they are a type of classifier that separates the genetic part sequence space into only two categories: sequences expected to have some amount of genetic part activity versus sequences expected to have minimal to none activity.

> **Note** The design constraints are not a quantitative prediction of functional activity; experimental characterization is still needed to validate designed parts. In general, it is advantageous to incorporate as much degeneracy into the constraints as possible to design larger toolboxes.

### Non-Repetitive σ<sup>70</sup> Promoters with CRISPRi with `Lmax=12`

We will first design $1000$ brand new promoters for constitutive transcription in prokaryotes, divided into two toolboxes. We want the first toolbox to have $500$ strong promoters, while for the second toolbox we want to design $500$ promoters with variable strength. Additionally, we want these promoters to be CRISPRi repressible for engineering system logic (see [Reis et al. (2019)](https://www.nature.com/articles/s41587-019-0286-9)). Importantly, we will use the findings from [Larson et al. (2013)](https://www.nature.com/articles/nprot.2013.132) to design our CRISPRi.

#### Designing the First Toolbox

The sequence constraint for the first toolbox is defined for strong constitutive transcription (consensus $-35$ and $-10$ hexamers, and an optimal spacing of $17$ bases separating the two). Additionally, a **PAM** is embedded in the $17$-bp spacer to repress transcription initiation via CRISPRi. To enhance initiation, we will also embed a G+C-rich motif into the insulating $20$-bp upstream region of $-35$ hexamer.

In [4]:
tb1_seq_constraint = ('S'*5    + # G+C-rich motif in Upstream (5 Bases)
                      'N'*15   + # Remaining 15 Bases in Upstream
                      'TTGACA' + # Consensus -35 Hexamer
                      'N'*6    + # First 6 Bases of 17-bp Spacer
                      'CCN'    + # PAM (Next 3 Bases of 17-bp Spacer)
                      'N'*8    + # Remaining 8 Bases of 17-bp Spacer
                      'TATAAT' + # Consensus -10 Hexamer
                      'N'*6)     # Discriminator

Let's review our sequence constraint.

In [5]:
print(tb1_seq_constraint)
print(' '*35 + '-'*20)
print(' '*35 + '{:^20}'.format('sgRNA Target Site'))

SSSSSNNNNNNNNNNNNNNNTTGACANNNNNNCCNNNNNNNNNTATAATNNNNNN
                                   --------------------
                                    sgRNA Target Site  


For promoters, we don't really have any DNA or RNA secondary structure constraint, so it can be all dots, that is, we don't care about the secondary structure anywhere along the sequence (this will change for downstream part design, as we'll see).

In [6]:
tb1_struct_constraint = '.'*len(tb1_seq_constraint)

In [7]:
tb1_struct_constraint

'.......................................................'

Once the design constraints are finalized, it is time to think about the experimental objectives as well. One possible objective might involve eliminating restriction sites from our promoters, so that we may clone them in successfully.

To do that, we can define some _model functions_ to help us generate promoters that are compatible for our cloning purposes. Importantly, while we can define functions to explicitly prevent our used cutsites (say, BamHI or XbaI, specifically), it would be better to prevent the occurence of any palindromic hexamer in our parts, which is usually a property of many restriction sites. That way, our promoters can be cloned and used in a variety of scenerios, without us having to restrict ourselves to using only the ones in which the specific restriction site motifs are absent, for a given cloning workflow.

> **Note** The `Non-Repetitive Parts Calculator` can optimize two types of functions for us - a **local model function** which is applied on a genetic part concurrently with addition of each nucleotide, and a **global model function** which is applied on a genetic part when it is fully constructed. The local model function must accept a single argument called `seq` (the partial sequence under construction) and returns either `(True, None)` if an evaluation was satisfied, or `(False, index)` where `index` is a traceback location where nucleotide choices need to be altered to fulfill an objective. The global model function also must accept a single input `seq` (a fully constructed sequence), and return either `True` if an objective was met, or `False` otherwise.

We will now develop a new objective function to prevent palindromic hexamers in our designed promoters to be evaluated concurrently as each new base is added to a promoter towards completion (a local model function). Our function will start evaluation when the **sixth** base (base at `index=5` or equivalently, when `len(seq)=6`) is added to a partial promoter under construction (necessary condition for evaluation), and guide the design process to steer clear of palindromic hexamers. We will also define any helper functions we need, and test our logic as we move forward.

In [8]:
comptab = str.maketrans('ATGC', 'TACG') # our string translation table for complementng strings

In [9]:
# helper function
def revcomp(seq):
    # reverse string, then return its complement
    return seq[::-1].translate(comptab)

In [10]:
assert revcomp('AAATTTGGGCCC') == 'GGGCCCAAATTT' # a quick test for revcomp function shows that it works

In [11]:
assert revcomp('GGGAAATTTCCC') == 'GGGAAATTTCCC' # another quick check on a palindrome confirms definition

In [12]:
# actual local model function
def prevent_cutsite(seq):
    # is our partial sequence long enough for evaluation?
    if len(seq) >= 6:
        # extract current (last-known) 6-mer for evaluation
        hexamer = seq[-6:]
        if hexamer == revcomp(hexamer): # a palindrome!
            state = False      # objective failed
            index = len(seq)-6 # we need to go back 6 bases to alter current hexamer
            return (state, index)
        else:
            return (True, None) # sequence is free of palindromic hexamers
    # otherwise, pass for shorter sequences
    else:
        return (True, None)

The optimization done by this function is straight-forward. We check if a partial sequence under construction ends with a palindromic hexamer. If it does, then we ask `Maker` to go back $6$ bases from our current index which is at index `len(seq)-6`, and start making alternate nucleotide choices starting **at** that location. If this function returns `True` for all locations starting at the sixth base, then naturally the complete part would be devoid of palindromic hexamers. Note also that we also return `True` for partial sequences shorter than $6$ bases.

`Maker` takes care of calling this function above with the addition of each base, when it is passed as the designated local model function. So, all we ever need to do inside this function is just evaluate the "current" case, i.e., evaluate the hexamer ending at the current index to ensure our optimization.

> **Note** The traceback index should point to the location _starting at which_ nucleotide choices need to altered. For example, in the illustrated `prevent_cutsite` function above, we need to go back $6$ bases from the current location if it is palindromic, and trace a new path through constraint space. This means, our traceback index should be `len(seq)-6=0` (the very beginning of our sequence) if our current sequence is of length $6$, and forms a palindromic hexamer.

In [13]:
assert prevent_cutsite('GGATCC') == (False, 0) # BamHI will be prevented

In [14]:
assert prevent_cutsite('TCTAGA') == (False, 0) # XbaI will be prevented

In [15]:
assert prevent_cutsite('GGGGGG') == (True, None)  # non-palindromic hexamer will pass our filter

At this point, we are ready to launch the `Non-Repetitive Parts Calculator` `Maker Mode` to design some promoters for us!

In [ ]:
# Record starting time
t0 = time.time()

# Execute Maker
toolbox1 = nrpcalc.maker(
    seed=1,                              # reproducible results
    seq_constr=tb1_seq_constraint,       # as defined above
    struct_constr=tb1_struct_constraint, # as defined above
    Lmax=12,                             # as stated in our goal
    target_size=500,                     # as stated in our goal
    part_type='DNA',                     # as stated in our goal
    local_model_fn=prevent_cutsite)      # as defined above

# Compute execution time
tf = time.time() - t0

In [17]:
print('Run took {:.2f}s'.format(tf)) # Show run time

Run took 0.64s


The console output shows that all our constraints passed initial checks, and were valid. `Maker` was then apparently able to design $500$ promoters for us based on all given constraints without failure in less than a second. Let's look at some of the parts produced, and compare it with our sequence constraint.

In [18]:
toolbox1[0] # show the first part designed

'CGGGCATTACTCCATTTGTCTTGACACTACACCCTACCTAGCCTATAATCCATGT'

In [19]:
tb1_seq_constraint # show the sequence constraint

'SSSSSNNNNNNNNNNNNNNNTTGACANNNNNNCCNNNNNNNNNTATAATNNNNNN'

In [20]:
toolbox1[499] # show the last part designed

'CCCGGACATCTCTTTCAATTTTGACACAACACCCCAACTGTAGTATAATAGTCGC'

The construction looks good, but it's always a good idea to verify explicitly if our design objectives were met. We will define a verification function to check if our completely constructed promoters are indeed devoid of palindromic hexamers.

In [21]:
def final_cutsite_check(seq):
    # grab all hexamers from the sequence
    hexamers = [seq[i:i+6] for i in range(len(seq)-6+1)]
    # if any of the hexamers are panlindromic, our
    # design objective was clearly not met!
    for hexamer in hexamers:
        if hexamer == revcomp(hexamer):
            return False # we will reject this part
    # None of the hexamers were palindromic!
    return True  # we will accept this part

In [22]:
# We will loop through the toolbox, and ensure
# all our parts pass the new global check
for promoter in toolbox1.values():
    assert final_cutsite_check(promoter) == True

Every promoter has passed our verification so our design objective for the first toolbox was met. As we will see in the next section, the evaluation function `final_cutsite_check` could have been specified to `Maker` directly via `global_model_fn` parameter, which would automatically execute the evaluation on a part, after it was completely designed, and accept/reject it accordingly.

The benefit of passing this check as a global model function to `Maker` is that the algorithm can adjust the number of trials it needs depending on an auto-estimated probability of evaluation failure.

#### Designing the Second Toolbox - First Attempt

For our second toolbox of promoters, we want to design $500$ variable strength promoters. Our sequence constraint will change accordingly.

In [23]:
tb2_seq_constraint = 'N'*20 + 'TTGNNN' + 'N'*6 + 'CCN' + 'N'*6 + 'WW' + 'WWWWWW' + 'N'*6
#                     -----    ------     -----   ---     ----------     ------     ----
#                     UPS      -35        SPACER  PAM     SPACER         -10        DIS

Notice, we introduced degeneracy in the $-35$ and opted for just weak bases (A/T)  in place of the $-10$ hexamer. Additionally, the $-10$ is also preceded by weak bases to potentially design promoters with various spacer region lengths ranging from $15$ to $17$ bp. We still retain the PAM in spacer for CRISPRi.

In [24]:
tb2_seq_constraint # review the constraint

'NNNNNNNNNNNNNNNNNNNNTTGNNNNNNNNNCCNNNNNNNWWWWWWWWNNNNNN'

Because things are more degenerate in our present sequence constraint, we might be interested in preventing cryptic hexamers within our promoters.

This is easily done with another local model function, that identifies if a hexamer elsewhere within our promoter under construction, has fewer mismatches when compared to the consensus motifs than the ones placed (by `Maker`) at the intended `-35` and `-10` locations.

In [25]:
# helper function 1
def hamming(x, y): # score mismatches between two strings
    return sum(x[i] != y[i] for i in range(min(len(x), len(y))))

In [26]:
assert hamming(x='000000', y='111111') == 6 # test case 1

In [27]:
assert hamming(x='000111', y='111111') == 3 # test case 2

In [28]:
# helper function 2
def cryptic_hexamer(cx, hx, dt): # returns True if hx is a cryptic hexamer
    '''
    cx = consensus motif for either -35 or -10
    hx = current hexamer under evaluation
    dt = number of mismatches between cx and current
         motif placed at -35 or -10
    '''
    # if current hexamer (hx) is closer to consensus
    # motif (cm) than the actual selected motif used
    # at the intended location (dt), then we have a
    # cryptic promoter under construction (True)
    if hamming(cx, hx) < dt:
        return True
    return False

In [29]:
assert cryptic_hexamer(cx='TTGACA', hx='AAAAAA', dt=3) == False # test case 1

In [30]:
assert cryptic_hexamer(cx='TTGACA', hx='TTGAGA', dt=3) == True  # test case 2

In [31]:
# actual local model function
def prevent_cryptic_promoter(seq, c35start, c10start, eval_index=None):
    '''
    seq - partial sequence under construction to be evaluated
    c35start - starting index of -35 hexamer (python indexing)
    c10start - starting index of -10 hexamer (python indexing)
    eval_index - the location ending at which a hexamer is to
                 be evaluated (default=None implies use the
                 last hexamer in current seq, i.e. ending at
                 len(seq))
    '''
    c35 = 'TTGACA' # defined -35 consensus
    c10 = 'TATAAT' # defined -10 consensus
    # sequence long enough to evaluate
    if len(seq) >= 6:
        # current index?
        end = len(seq)
        
        # which hexamer to evaluate?
        if eval_index is None: # no eval_index provided
            eval_index = end   # use the hexamer ending at current index (end)
        # otherwise use appropriate eval_index provided
        
        # extract current / appropriate hexamer
        hx = seq[eval_index-6:eval_index]
        
        # Case: -35 hexamer
        
        # current hexamer is at -35 or -10 location?
        # then skip evaluation for extracted hexamer
        if end == c35start+6:
            return (True, None) # skip -35 location
        if end == c10start+6:
            return (True, None) # skip -10 location
        
        # extract current -35 hexamer
        # if there is one
        s35 = None
        if end > c35start+6: # a -35 motif has been placed
            s35 = seq[c35start:c35start+6]
        
        # set -35 hexamer cutoff
        if s35 is None: # no -35 hexamer present yet
            d35 = 3 # default distance to prevent
        else:
            d35 = hamming(s35, c35) # actual distance to prevent
        
        # evaluate hx for -35 hexamer
        if cryptic_hexamer(cx=c35, hx=hx, dt=d35):
            return (False, end-6) # our current hexamer is a cryptic -35;
                                  # go back 6 bases
        
        # Case: -10 hexamer
        
        # extract current -10 hexamer
        # if there is one
        s10 = None
        if end > c10start+6:
            s10 = seq[c10start:c10start+6]
        
        # set -10 hexamer cutoff
        if s10 is None: # no -10 hexamer present yet
            d10 = 3 # default distance to prevent
        else:
            d10 = hamming(s10, c10) # actual distance to prevent
        
        # evaluate hx for -35 hexamer
        if cryptic_hexamer(cx=c10, hx=hx, dt=d10):
            return (False, end-6) # our current hexamer is a cryptic -10;
                                  # go back 6 bases
        
        # both -35 and -10 checks passed
        return (True, None) # part is OK
    
    # not long enough to evaluate
    return (True, None) # part is OK .. so far

In [32]:
# test case 1 - a partial sequence with last 6 bases very similar to the -35 consensus
assert prevent_cryptic_promoter(seq='GGGGGGGGTTGACT', c35start=20, c10start=20+6+17) == (False, 8)

In [33]:
# test case 2 - a partial sequence with last 6 bases very similar to the -10 consensus
assert prevent_cryptic_promoter(seq='GGGGGGGTATAGT', c35start=20, c10start=20+6+17) == (False, 7)

In [34]:
# test case 3 - a partial sequence with last 6 bases dissimilar to the both motifs
assert prevent_cryptic_promoter(seq='GGGGGGGGAAGATC', c35start=20, c10start=20+6+17) == (True, None)

The above local model function `prevent_cryptic_promoter` utilizes many smaller functions in order to make its evaluation, which is fine. The only thing we need to take care of in order to use this function with `Maker` is the setting of `c35start` and `c10start` parameters, which are required for the function to work (note `eval_index` has a default value of `None` so we need not worry about it right now), given that `Maker Mode` only works with local model functions that just takes in a single input - the partial sequence under construction (`seq`).

Some obvious solutions would be to hardcode all parameters apart from `seq` inside the local model function, or give them default values like we did for `eval_index`, but we don't really need to do either. Instead, we have two more options.

The first option is to define a lambda function to wrap the above function like so.

In [35]:
prevent_cryptic = lambda seq: prevent_cryptic_promoter(seq=seq, c35start=20, c10start=43)

We could then set `local_model_fn=prevent_cryptic`, and our optimization would come through. This wrapping leaves the underlying function `prevent_cryptic_promoter` free for more general use later. For example, it could be used to power an additional global model function that checks the completely constructed part for cryptic hexamers by re-evaluating the hexamers at each index starting at $5$ via the `eval_index` parameter (more on this a little later).

The second option is to define a wrapper function explicitly.

In [36]:
def prevent_cryptic(seq):
    return prevent_cryptic_promoter(
        seq=seq,
        c35start=20,
        c10start=43)

Notice, that now we actually have two local model functions so far: (1) the `prevent_cryptic` function and (2) the previously used `prevent_cutsite` function used for the first toolbox. In such multi-objective design scenerios, we would have to write a meta local model function, which would run these individual local model functions along with any specific parameters. Here's an example:

In [37]:
# a meta local model function
def variable_promoter_local(seq):
    # check the first objective
    outcome,index = prevent_cutsite(seq)
    # return traceback index if objective fails
    if outcome == False:
        return False,index
    # check the second objective
    outcome,index = prevent_cryptic_promoter(
        seq=seq,
        c35start=20,
        c10start=43)
    # return traceback index if objective fails
    if outcome == False:
        return False,index
    # every objective met .. good to go!
    return (True, None)

> **Note** It is important to be careful about string indexing when the function logic becomes moderately complex. For example, in `Python` strings are $0$-indexed, which means that the $-35$ hexamer starts after the first $20$ bases in the upstream region at index $20$ (i.e. the $21$st base belongs to $-35$). It is also important to note when a model function should be evaluated. For example, if the evaluation logic requires at least an $8$-bp sequence, parts shorter than that length should be evaluated `True` to let the sequence grow long enough for evaluation.

> **Note** When there are multiple local objectives at play that evaluate properties of the partial sequence at an upstream location, it is often advantageous to return the traceback index that occurs earlier in the sequence position. For example, one thing we could do in a meta local model function is evaluate two objectives, and return `(False, min(index1, index2))` if both objective functions failed with different traceback locations, or just `(False, index1)` if only the first one failed and so on. We could also weigh the various objectives differently, and choose to return the most important traceback index first, rather than the second most important traceback index etc.

> **Note** It is possible to embed external models as evaluators into the `Non-Repetitive Parts Calculator`. For example, rather than only preventing cryptic motifs as shown above, we could have also used a `scikit-learn` `Lasso` model, as described in [our publication](https://www.nature.com/articles/s41587-020-0584-2), to design promoters within specific dynamic ranges. We would load the model (unpickle) into memory, and for every promoter completely constructed by `Maker`, we would evaluate the predicted initation rate and only accept parts that satisfied our criteria (a global model function). Alternatively, we could further identify, using the model, which of the components (hexamers, spacer GC etc.) prevented a part from being accepted, and returned a traceback index accordingly (i.e. converted the global into a local model function) to explore nucleotide choices concurrently with part design.

> **Note** It is always a good idea to test the individual functions called by a meta local or global model function, using simple cases. Notice, how we have `assert`-ed a few test cases for the helper functions above.

> **Note** If `Maker` takes an impossible amount of time to create a single part in the prescence of a model function, it is worth investigating if the model function in context gets stuck in an infinite loop or an edge case. A quick check would be to run `Maker` with all constraints except the model function. If `Maker` is able to design parts quickly in the absence of the model function, then the slow-down is naturally due to the the model function itself, which should be investigated and optimized.

Now that our meta local model function is ready, we can define a meta global model function that calls `final_cutsite_check` as well as `prevent_cryptic_promoter` like so.

In [38]:
# a meta global model function
def variable_promoter_global(seq):
    # check for cutsites post construction
    if not final_cutsite_check(seq):
        return False # cutsites found!
    
    # note: the following block could be
    # its own function, and called by this
    # meta global function
    
    # check for cyptic hexamers
    # starting at the 6th base
    for eval_index in range(6, len(seq)):
        # use the generalized evaluation function
        state, index = prevent_cryptic_promoter(
            seq=seq,
            c35start=20,
            c10start=43,
            eval_index=eval_index)
        # there is a cryptic hexamer ending
        # at the current location
        if state is False:
            return False
    
    # all checks passed!
    return True

With all our evaluators completed, we're ready to design our second toolbox of promoters. Let's call upon `Maker` to do our bidding.

In [ ]:
# Record starting time
t0 = time.time()

# Execute Maker
toolbox2_attempt1 = nrpcalc.maker(
    seed=2,                                    # reproducible results
    seq_constr=tb2_seq_constraint,             # as defined above
    struct_constr=tb1_struct_constraint,       # same as toolbox1
    Lmax=12,                                   # as stated in our goal
    target_size=500,                           # as stated in our goal
    part_type='DNA',                           # as stated in our goal
    local_model_fn=variable_promoter_local,    # as defined above
    global_model_fn=variable_promoter_global)  # as defined above

# Compute execution time
tf = time.time() - t0

In [40]:
print('Run took {:.2f}s'.format(tf)) # Show run time

Run took 3.46s


Notice, that the running time increased from less than one second for the first toolbox, to about four seconds for this present toolbox. This is because the running time of `Maker` greatly depends on the complexity of the underlying model functions. For the second toolbox, we used both local and global model functions, each of which considered two sub-objectives inside them. `Maker` was able to satisfy all of these objectives and finished in under four seconds.

Let's review our newly minted toolbox!

In [41]:
toolbox2_attempt1[0] # first promoter in second toolbox

'TCACCTAGCGCAGCGGTCAGTTGTACCGGGTCCCCCGCGTTTTTTTAATTCGATT'

In [42]:
toolbox2_attempt1[499] # last promoter in second toolbox

'CGACACGAAACTACGCGCCATTGTGTTCCTACCCGAGGAACAAAAAAAAGGACGC'

#### Designing the Second Toolbox - Second Attempt

The reason we called the previous sub-section a **"First Attempt"** is because, the second toolbox we designed above is non-repetitive to itself, but not against `toolbox1` promoters designed apriori. To verify non-repetitiveness in construction, we can use `Finder Mode`.

In [43]:
# combine both toolboxes
promoters = list(toolbox1.values())
promoters.extend(toolbox2_attempt1.values())

In [ ]:
# compute the number of non-repetitive promoters
non_repetitive_promoters = len(nrpcalc.finder(
    seq_list=promoters,
    Lmax=12))

In [45]:
assert non_repetitive_promoters < 1000 # we're short of our goal ... some promoters were repetitive

As we can see, the final non-repetitive toolbox when both toolboxes are combined together has less than $1000$ parts in it, which is short of our intended goal. This is where concept of **"background"** comes into play (check [DOCS](https://github.com/ayaanhossain/nrpcalc/blob/master/docs/DOCS.md) for `background` API details).

> **Note** `Finder` is an unstable algorithm by design. What this means is that, in a scenario consisting of a list of repetitive parts, the returned non-repetitive subset is approximately the largest possible non-repetitive toolbox, but this may change slightly across multiple runs for the same set of inputs. There is no way known, generally, to be certain of what the absolutely largest toolbox size actually is (unless all parts were either repetitive or non-repetitive as in the case during verification of parts returned by `Maker`), so we retained a level of stochasticity in `Finder`. This encourages us to run `Finder` (which is pretty fast in practice) several times on a candidate toolbox of parts, and then select the largest non-repetitive toolbox returned across all runs.

To create the second toolbox while ensuring it is non-repetitive to the first one, we will populate a temporary `background` object.

In [46]:
bkg = nrpcalc.background(
    path='./tmp_bkg', # we store the background on disk in the 'tmp_bkg' directory on current path
    Lmax=12)          # same Lmax as toolbox1

In [47]:
bkg # checking background path and content, we see it has zero elements

kmerSetDB stored at ./tmp_bkg/ with 0 13-mers

In [48]:
# # we could add the promoters one-by-one
# for promoter in toolbox1.values():
#     bkg.add(promoter)

# or add it all in one-shot
bkg.multiadd(toolbox1.values())


[Background Processing]


  Adding Seq 268: GCCGCCTGTC...

  Adding Seq 499: CCCGGACATC...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [49]:
bkg # now, background is populated with toolbox1 k-mers

kmerSetDB stored at ./tmp_bkg/ with 21500 13-mers

With our `background` populated, we are now ready to design our actual second toolbox such that it is indeed non-repetitive to the first toolbox, therefore allowing both toolboxes to be used simultaneously. This is what we refer to as **toolbox chaining**. For example, you can chain a `Maker` job against a genome inserted in `background`. You can also use `background` with `Finder` jobs to incrementally enlarge a central collection of parts from multiple sources.

In [ ]:
# Record starting time
t0 = time.time()

# Execute Maker
toolbox2 = nrpcalc.maker(
    seed=3,                                    # reproducible results
    seq_constr=tb2_seq_constraint,             # as defined above
    struct_constr=tb1_struct_constraint,       # same as toolbox1
    Lmax=12,                                   # as stated in our goal
    target_size=500,                           # as stated in our goal
    part_type='DNA',                           # as stated in our goal
    local_model_fn=variable_promoter_local,    # as defined above
    global_model_fn=variable_promoter_global,  # as defined above
    background=bkg)                            # as defined above

# Compute execution time
tf = time.time() - t0

In [51]:
print('Run took {:.2f}s'.format(tf)) # Show run time

Run took 4.19s


Notice, that the running time now increased from less than four seconds in our previous attempt to slightly more than four seconds in our current attempt. This is because we introduced `background` as an additional constraint for the design job above.

Let's update our `background` and use `Finder` again to verify our construction of the second toolbox.

In [52]:
bkg.multiadd(toolbox2.values()) # updated with second toolbox for building next set of parts


[Background Processing]


  Adding Seq 268: CCAACATCTG...

  Adding Seq 499: AAGACTGCAC...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [53]:
# recreate the list of promoters for evaluation
promoters = list(toolbox1.values()) + list(toolbox2.values())

In [ ]:
# assess non-repetitiveness
non_repetitive_promoters = len(nrpcalc.finder(
    seq_list=promoters,
    Lmax=12))

In [55]:
assert non_repetitive_promoters == 1000 # no promoters missing!

### Non-Repetititve Ribosome Binding Sites with `Lmax=14`

To complement our designed promoter toolboxes, we will next design a toolbox of prokaryotic ribosome binding sites (RBSs). We will primarily be using findings from [Salis et al. (2009)](https://www.nature.com/articles/nbt.1568) for designing our RBSs.

We aim to design $1000$ _de novo_ RBS sequences that are non-repetitive to our promoter toolboxes designed in the previous sections. Our RBS sequence constraint is therefore highly degenerate, containing a $26$-bp upstream region, a $4$-bp standby site, and a $9$-bp consensus Shine-Dalgarno (SD) motif ('UAAGGAGGA') separated from the start codon ('AUG') by a near-optimal $6$-bp spacer. Importantly, the structural constraint specifies that there will be a small hairpin on the $5'$-end of designed sequences to insulate the RBS against the formation of undesired structures that might inhibit ribosome binding to the Shine-Dalgarno motif.

Let's define and review our constraints.

In [56]:
tb3_seq_constraint = 'N'*26 + 'N'*4 + 'UAAGGAGGA' + 'N'*6 + 'AUG'
#                     -----    ----    ---------     ----    ---
#           Upstream Region    SBS     SD Motif    SPACER  START

In [57]:
tb3_struct_constraint = '.(((((....)))))...............xxxxxxxxxxxxxxxxxx'

In [58]:
print(tb3_seq_constraint)
print(tb3_struct_constraint)

NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNUAAGGAGGANNNNNNAUG
.(((((....)))))...............xxxxxxxxxxxxxxxxxx


The dots (`.`) in the structure constraint implies that the bases in the sequence constraint at the corresponding locations are free to either base-pair or not when a candidate part is generated. Bases marked with parenthesis (`(` and `)`) indicate that the folded structure must contain those designated base-pairings, for example the second base must pair with the fifteenth base and so on. Bases marked with `x` are forbidden from being part of any base pairing in the secondary `RNA` structure. This _dot-parenthesis-x_ notation is inspired from the secondary structure notation used by nucleic acid structure prediction programs such as `ViennaRNA`.

Before we design the RBS toolbox, we must note that the constraint for RBS toolbox here includes an `Lmax` of $14$, whereas, the promoters were designed with an `Lmax` of $12$ bases. This is because, there is a big $9$-bp constant Shine-Dalgarno motif in the sequence constraint which doesn't leave too many $13$-mers (recall `Lmax=12`) for constructing thousands of non-repetitive RBSs. As proof, let's try constructing the RBS toolbox with `Lmax=12`, without using any `background` developed previously and only using the fast `mfe` (minimum free energy) structure evaluation (a relaxed design scenerio).

In [ ]:
# Record starting time
t0 = time.time()

# Execute Maker
toolbox3_attempt1 = nrpcalc.maker(
    seed=4,                                    # reproducible results
    seq_constr=tb3_seq_constraint,             # as defined above
    struct_constr=tb3_struct_constraint,       # as defined above
    Lmax=12,                                   # as stated in our goal
    target_size=1000,                          # as stated in our goal
    part_type='RNA',                           # as stated in our goal
    struct_type='mfe',                         # as defined above        
    local_model_fn=None,                       # as defined above
    global_model_fn=None,                      # as defined above
    background=None)                           # as defined above

# Record execution time
tf = time.time() - t0

In [60]:
print('Toolbox Size reached = {}, but Target Size was 1000'.format(
    len(toolbox3_attempt1)))

Toolbox Size reached = 218, but Target Size was 1000


In [61]:
print('Run took {:.2f}s'.format(tf)) # Show run time

Run took 503.08s


As we can see in the output, `Maker` first warned us that it might not be able to make $1000$ parts as specified in the `target_size`, but it ventured forth, taking about nine minutes to explore the design space and constructing $200+$ RBSs, before giving up.

In [62]:
assert ('''
 Only 4 bases free in the last 13 bp window
           containing the complete SD motif
                              -------------
                              |||||||||****
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNUAAGGAGGANNNNNNAUG
                          ****|||||||||
                          -------------
      True for the first window as well
''')

The $9$-bp constant motif in sequence constraint leaves only $4$ degenerate bases in every $13$-bp window containing the complete SD sequence, implying at most $4^4 = 256$ possible parts for the given sequence constraint. Such _k_-mer windows which limit the overall design space and prevents the reaching of `target_size` are called **`Lmax` limiting windows**. `Maker` was able to design $218$ of the maximum possible toolbox size before failing to find suitable _k_-mers for making newer RBSs. If we wanted, we could try increasing the `jump_count` and/or `fail_count` (see [API](https://github.com/ayaanhossain/nrpcalc/blob/master/docs/DOCS.md)) to try to reach all $256$ of the possible RBSs, although the severe structure constraint might prevent selection of some _k_-mers to realize all of these sequences.

Our goal, however, is to build $1000$ RBSs which is clearly not possible given an `Lmax` of $12$ for the specified sequence constraint. We could try introducing more degeneracy into the SD motif which might relax our constraints enough to fix the issue. But, if we don't want to alter the motif, we would have to increase our `Lmax` to expand our design space. An `Lmax` of $14$ seems reasonable, giving `Maker` $4^6 = 4096$ possible _k_-mer selection choices for all $15$-bp windows encompassing the SD motif.

Now, that we've decided to use an `Lmax=14` for our toolbox, how do we unify our present RBS toolbox with the previously designed promoter toolboxes with `Lmax=12`, in terms of non-repetitiveness? It is a feature of the `Non-Repetitive Parts Calculator` that one can use a `background` initialized with a lower `Lmax` for a design job specifying a higher `Lmax`. So, if designing RBSs was our last job, it would be legal and recommended to use `bkg` as the `background`, since its `Lmax=12`, and our new RBS toolbox would be built with an `Lmax` of $14$ (higher). The opposite scenerio is not supported by `Maker` for algorithmic efficiency reasons. This implies that toolbox chaining can progressively move from a lower `Lmax` to a higher one, but not the other way round.

Alternative approaches include (1) initializing a new `background` with `Lmax=14` and inserting all previous promoters into it, followed by using the new `background` for designing RBSs, or (2) defining a new local model function that prevents every $13$-mer in the new RBSs under construction from coinciding with the previous `background` (`bkg`) containing the $13$-mers from the promoters.

The first alternative solution is pretty straight-forward, and the one we'll use for designing our toolboxes in the subsequent sections (since we'd be permanently moving onto an `Lmax=14` for present and subsequent toolboxes, and `bkg` due to its `Lmax=12` wouldn't be appropriate there), but for our current RBS toolbox design we'll look at the second alternative option to see an example of a model function that works with `background` objects.

Let's look at a possible local model function for achieving the second alternative option.

In [63]:
def prevent_promoter_conflict(seq):
    # evaluation criteria met?
    if (len(seq)) >= 13:
        # extract k-mer
        kmer = seq[-13:]
        # check for conincidence
        if kmer in bkg: # k-mer conflict found
            return (False, len(seq)-13) # retrace path
        # no conflict
        return (True, None)
    # too short a sequence
    else:
        return (True, None)

In [64]:
# toolbox1 promoter fails our evaluation as expected
assert prevent_promoter_conflict(seq=toolbox1[0]) == (False, len(toolbox1[0])-13)

In [65]:
# toolbox2 promoter also fails our evaluation as expected
assert prevent_promoter_conflict(seq=toolbox2[499]) == (False, len(toolbox2[0])-13)

In [66]:
# a poly-G 13-mer was absent in the promoters, so it is OK to be used for the RBSs
assert prevent_promoter_conflict(seq='G'*13) == (True, None)

Our local model function is done, and our `Lmax` is revised to $14$. However, unlike the previous RBS toolbox design attempt, instead of relying on just `mfe` for structure evaluation, we'll use `mfe` + `centroid` = `both` as our `struct_type` parameter to ensure both `mfe` and `centroid` conform to the given structure constraint. This ensures that designed parts fold into a given structure with high probability (at the cost of slightly increased computation time).

In [ ]:
# Record starting time
t0 = time.time()

# Execute Maker
toolbox3 = nrpcalc.maker(
    seed=5,                                    # reproducible results
    seq_constr=tb3_seq_constraint,             # as defined above
    struct_constr=tb3_struct_constraint,       # as defined above
    Lmax=14,                                   # as revised from our previous attempt
    target_size=1000,                          # as stated in our goal
    part_type='RNA',                           # as stated in our goal
    struct_type='both',                        # as revised from our previous attempt
    local_model_fn=prevent_promoter_conflict,  # as defined above
    global_model_fn=None,                      # none required
    background=None)                           # background conflict resolved via local model

# Compute execution time
tf = time.time() - t0

In [68]:
print('Run took {:.2f}s'.format(tf)) # Show run time

Run took 271.39s


Compared to the previous attempt, the present attempt finishes in less than five minutes and we designed exactly $1000$ non-repetitive RBSs per our goal.

Let's review the parts, and verify non-repetitiveness of the new toolbox against `bkg` by specifying it as a background to `Finder`. If `Finder` returns all $1000$ parts as non-repetitive, then our model function worked as intended despite us not using `bkg` as the `background` directly in our call to `Maker`.

In [69]:
toolbox3[0] # first RBS designed

'UGGUUCUAACGAGCCGAUCAGUUUAAUGUAUAAGGAGGAAGCCCAAUG'

In [70]:
toolbox3[999] # last RBS designed

'CACCGGGUGACUGGUUGGCAACGCGUUAUAUAAGGAGGACUACUAAUG'

In [71]:
assert len(nrpcalc.finder(
    seq_list=toolbox3.values(),
    Lmax=14,
    verbose=False,  # just assert
    background=bkg)) == 1000 # our goal of 1000 brand new non-repetitive RBSs is achieved!

### Non-Repetitive Toehold Switches with `Lmax=14`

We will now design $1000$ non-repetitive toehold RNA switches for programmable protein expression. Our constraints for designing these toehold switches are based on the work of [Green et al. (2014)](https://www.sciencedirect.com/science/article/pii/S0092867414012896).

Before we embark on the design, let's delete the previous `background` (`bkg`) and initialize a new `background` with `Lmax=14`, into which we'll insert all of the previous toolboxes, so that we can use it for designing our toehold switches non-repetitive to all previously designed parts.

In [72]:
bkg.drop() # deletes the background from disk

True

In [73]:
chained_bkg = nrpcalc.background(
    path='./chained_bkg', # a new background path
    Lmax=14)              # updated Lmax

In [74]:
chained_bkg # check new background path and content

kmerSetDB stored at ./chained_bkg/ with 0 15-mers

In [75]:
chained_bkg.multiadd(toolbox1.values()) # add the first promoter toolbox
chained_bkg.multiadd(toolbox2.values()) # add the second promoter toolbox
chained_bkg.multiadd(toolbox3.values()) # add the third toolbox containing RBSs


[Background Processing]


  Adding Seq 268: GCCGCCTGTC...

  Adding Seq 499: CCCGGACATC...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 268: CCAACATCTG...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 499: AAGACTGCAC...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 268: GCGACAGGUC...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 533: UGUCCAGAGG...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 798: GGGUCAGGGA...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 999: CACCGGGUGA...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [76]:
chained_bkg # review background post insertion

kmerSetDB stored at ./chained_bkg/ with 75000 15-mers

Our constraint for toehold switches primarily includes a hairpin loop, and contains a $30$-bp **trigger RNA** sequence usptream of an embedded $7$-bp consensus Shine-Delgarno motif ('AGGAGGA'), separated from the start codon ('AUG') by the remaining $6$-bp stem of domain _B_, and ends with a $21$-bp linker sequence. Notably, everything upstream of the linker portion of the design has a very specific structure requirement.

Let's define the sequence and structure constraints and review them.

In [77]:
tb4_seq_constraint = 'N'*12 + 'N'*9 + 'N'*3 + 'N'*6 + 'AGGAGGA' + 'N'*6 + 'AUG' + 'N'*9 + 'N'*21
#                     -----    --------------------    -------     --------------------    -----
#                  Domain A     Domain B with Bulge    SD Motif    Domain B with START     LINKER

In [78]:
tb4_struct_constraint = 'xxxxxxxxxxxx(((((((((xxx((((((xxxxxxx))))))xxx))))))))).....................'

In [79]:
print('{:^30}'.format('Trigger RNA Sequence'))
print('-'*30)
print(tb4_seq_constraint)
print(tb4_struct_constraint)

     Trigger RNA Sequence     
------------------------------
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNAGGAGGANNNNNNAUGNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
xxxxxxxxxxxx(((((((((xxx((((((xxxxxxx))))))xxx))))))))).....................


Although, at first glance, the constraints look fine and degenerate enough, there is a potential pitfall waiting for us when we feed these constraints to `Maker`.

Notice, that the $7$-bp SD motif ('AGGAGGA') is flanked by domain _B_ bases on either side, which creates a $15$-bp _k_-mer window with 'AGGAGGA' in the middle and four paired bases on either side. This is illustrated below.

In [80]:
assert ('''
                                15bp
                          ---------------
                                7bp
                          4bp ||||||| 4bp
                          ||||       ||||
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNAGGAGGANNNNNNAUGNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
xxxxxxxxxxxx(((((((((xxx((((((xxxxxxx))))))xxx))))))))).....................
                          ||||       ||||
                          4bp ||||||| 4bp
                                7bp
                          ---------------
                                15bp
''')

In this $15$-mer window, the last four bases are always going to be complementary to the first four bases. So, as soon as the first four bases ('N's) are filled in by `Maker`, the fate of the last four bases are automatically determined (they will be complementary to the first four bases). The $7$-bp SD motif is a constant in our sequence constraint which leaves only the first four bases to be selected variably by `Maker`, resulting in the last four bases becoming a dynamically inserted constant for each imaginable run. Thus, instead of working with a degenerate $15$-bp window with $7$ bases fixed, we're actually working with a $15$-bp window with $7+4=11$ bases fixed. This leaves us with $4^4 = 256$ possible nucleotide combinations to fill up this window, implying, a theoretical maximum toolbox size of only $256$ toehold switches. Our goal for $1000$ non-repetitive toehold switches, even with `Lmax=14`, will not be fulfilled given how we have framed the sequence and structure constraint.

Rather than abandoning hope, we may go back to the original paper for insights. It is clear that an RBS must be located between the two halves of domain _B_, but should this RBS just consist of a consensus $7$-bp SD motif only? The motif can potentially be "padded" on either side, and still leave us with effective RBSs. Accordingly, we will modify our sequence constraint to pad the SD motif with three 'N's on the $5'$-end, while ensuring that those bases remain unpaired via our modified structure constraint. This expands our design space by sixty four fold, making our goal of $1000$ non-repetitive toehold switches a real possibility. Let's re-define the constraints, and review them one last time.

In [81]:
tb4_seq_constraint = 'N'*12 + 'N'*9 + 'N'*3 + 'N'*6 + 'NNNAGGAGGA' + 'N'*6 + 'AUG' + 'N'*9 + 'N'*21
#                     -----    --------------------    ----------     --------------------    -----
#                  Domain A     Domain B with Bulge    Padded SD      Domain B with START     LINKER

In [82]:
tb4_struct_constraint = 'xxxxxxxxxxxx(((((((((xxx((((((xxxxxxxxxx))))))xxx))))))))).....................'

In [83]:
print('{:^30}'.format('Trigger RNA Sequence'))
print('-'*30)
print(tb4_seq_constraint)
print(tb4_struct_constraint)

     Trigger RNA Sequence     
------------------------------
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNAGGAGGANNNNNNAUGNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
xxxxxxxxxxxx(((((((((xxx((((((xxxxxxxxxx))))))xxx))))))))).....................


Because we're dealing with toehold switches, it is imperative to prevent any start codon in the stem separating the SD motif and the designated start codon as well any start or stop codons after the designated start codon. Time for a quick local model function.

> **Note** `Maker` builds and returns either **DNA** or **RNA** strings depending on the input `part_type` specification. The `part_type` is used to ensure correct base pairing, and select the correct energy parameters for evaluating the structure constraint for the intended scenerio. For example, toehold RNA switches are designed using correct parameters so that when they finally fold in their RNA state, they have the correct conformation. This also means that all local and global model functions used for optimization should evaluate **DNA** or **RNA** strings for evaluation depending on the `part_type` for sake of correctness.

In [84]:
start_codon = 'AUG' # as opposed to 'ATG'

In [85]:
stop_codons = set(['UAG', 'UAA', 'UGA']) # all stop codons are defined

In [86]:
def prevent_codon(seq):
    # we don't evaluate if were're at or before SD motif, or at
    # the designated start codon location and the the two bases
    # right after the start codon (which do not form an in-frame codon)
    
    # case 1: at or before SD motif
    if len(seq) <= 40: # pass evaluation
        return (True, None)
    
    # case 2: at the designated start codon or the two
    # bases right next to it
    if 47 <= len(seq) <= 49+2: # pass evaluation
        return (True, None)
    
    # actual evaluation time!
    
    # case 1: we have entered in the stem between SD and start codon
    if 41 <= len(seq) <= 46:
        # extract codon candidate
        cdn = seq[-3:]
        # is this a start codon?
        if cdn == start_codon:
            return (False, len(seq)-3) # go back three places
        # not a start codon
        return (True, None)
    
    # case 2: we have entered the linker region beyond the start codon
    if len(seq) >= 52: # first in-frame codon after the start codon
        # extract codon candidate
        cdn = seq[-3:]
        # is the codon candidate a stop codon?
        if cdn in stop_codons:
            return (False, len(seq)-3) # go back three places
        # is the codon candidate a start codon?
        if cdn == start_codon:
            return (False, len(seq)-3)
        # candidate is not a stop codon
        return (True, None) # pass

In [87]:
assert prevent_codon(seq='A'*25) == (True, None) # short sequences pass

In [88]:
assert prevent_codon(seq='A'*40 + 'AUG') == (False, 40) # start codon in spacer after SD prevented

In [89]:
assert prevent_codon(seq='A'*46 + 'AUG') == (True, None) # start codon at designated location not evaluated

In [90]:
assert prevent_codon(seq='A'*50 + 'UAA') == (False, 50) # stop codons after start codon prevented

In [91]:
assert prevent_codon(seq='A'*50 + 'GCA') == (True, None) # other codons are fine

Let's now build our brand new toolbox of non-repetitive toehold switches and review them!

In [ ]:
# Record starting time
t0 = time.time()

# Execute Maker
toolbox4 = nrpcalc.maker(
    seed=6,                                    # reproducible results
    seq_constr=tb4_seq_constraint,             # as defined above
    struct_constr=tb4_struct_constraint,       # as defined above
    Lmax=14,                                   # as defined above
    target_size=1000,                          # as stated in our goal
    part_type='RNA',                           # as stated in our goal
    struct_type='both',                        # as stated in our goal
    local_model_fn=prevent_codon,              # as defined above
    global_model_fn=None,                      # no requirement of a global check
    background=chained_bkg)                    # as defined above

# Compute execution time
tf = time.time() - t0

In [93]:
print('Run took {:.2f}s'.format(tf)) # Show run time

Run took 669.34s


In [94]:
toolbox4[0] # first switch in the toolbox

'UUGUCACACUACAAAGAAAGCGUAGUGACGAUUAGGAGGACGUUGCAUGGCUUUCUUUACAAUCUGUCGUCGCUACCGU'

In [95]:
toolbox4[999] # last switch in the toolbox

'GUGUAUCGUUAGCCUUUCCUCAGAUGGAAAGGAAGGAGGAUUUCCGAUGGAGGAGAGGGAGGUUCAAAUCUUGUUGUGU'

In [96]:
assert len(nrpcalc.finder(
    seq_list=toolbox4.values(),
    Lmax=14,
    verbose=False, # just assert
    background=chained_bkg)) == 1000 # job done!

### Non-Repetitive Intrinsic Terminators with `Lmax=14`

For our final demonstration, we will design $1000$ non-repetitive rho-independent bacterial terminators based on the works of [Chen et al. (2013)](https://www.nature.com/articles/nmeth.2515), and [Nielsen et al. (2016)](https://science.sciencemag.org/content/352/6281/aac7341)

Our design includes a highly degenerate sequence constraint with embedded poly-A and poly-U motifs, and a $15$-bp stem in the structure constraint. Based on the paper, the $8$-bp U-rich tract is $8$ bases downstream of the stem, and pairs with the complementary A-rich tract immediately upstream of the stem. We will not be using any model functions in this example, but we will ensure that the terminators are non-repetitive to all of the toolboxes designed above.

In [97]:
#                                A Tract|Strong Bases             Strong Bases         U Tract   
#                                -------|-----                          ------        --------   
tb5_seq_constraint    = 'NNNNNNNNAAAAAAAASNSNSNNNNNNNNNNNNNNNNNNNNNNNNNNNSNSNSNNNNNNNNUUUUUUUUNNNNNNNN'
tb5_struct_constraint = '........(((((((((((((((((((((((xxxxxxx)))))))))))))))xxxxxxxx))))))))........'
#                                        ---------------
#                                        15-bp Stem

Note, that the terminator structure constraint mandates a $15$-bp stem which implies that all designed terminators must have an internal repeat of $15$ bases, yet our desired `Lmax` is $14$. In such scenerios, we can set `internal_repeats=True`, and ask `Maker` to preserve parts with internal repeats, while still eliminating shared repeats between all pairs of parts.

Let's update our `background` object with the previously designed toehold switches, and then design the terminators.

In [98]:
chained_bkg.multiadd(toolbox4.values()) # update background with toehold switches


[Background Processing]


  Adding Seq 268: UAUCUACUGU...

  Adding Seq 533: AUAUCCACCU...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 798: AUCACGAAUU...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

  Adding Seq 999: GUGUAUCGUU...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [ ]:
# Record starting time
t0 = time.time()

# Execute Maker
toolbox5 = nrpcalc.maker(
    seed=7,                                    # reproducible results
    seq_constr=tb5_seq_constraint,             # as defined above
    struct_constr=tb5_struct_constraint,       # as defined above
    Lmax=14,                                   # as defined above
    internal_repeats=True,                     # as stated in our goal
    target_size=1000,                          # as stated in our goal
    part_type='RNA',                           # as stated in our goal
    struct_type='both',                        # as stated in our goal
    local_model_fn=None,                       # no requirement of a local check
    global_model_fn=None,                      # no requirement of a global check
    background=chained_bkg)                    # as defined above

# Compute execution time
tf = time.time() - t0

In [100]:
print('Run took {:.2f}s'.format(tf)) # Show run time

Run took 821.34s


Let's review our designed terminators, and ensure that all toolboxes are non-repetitive to each other, as a final check.

In [101]:
toolbox5[0] # first terminator designed

'UAGAGUGCAAAAAAAAGCGUGUACAUCCGAAAUGUUGCUUCGGGUGUACACGCAGAAGUUAUUUUUUUUGGCGGAGG'

In [102]:
toolbox5[999] # last terminator designed

'UACGGUCAAAAAAAAAGUCACUAGUUGUCGUGUAGAUAGCGGCGGCUAGUGACAUGUCAACUUUUUUUUUCAAAUGA'

In [103]:
all_toolboxes = [] # our final toolbox list
# insert all toolboxes designed so far into all_toolboxes
for toolbox in [toolbox1, toolbox2, toolbox3, toolbox4, toolbox5]:
    all_toolboxes.extend(toolbox.values())

In [104]:
assert len(nrpcalc.finder(
    seq_list=all_toolboxes,
    internal_repeats=True,  # allow internal repeats due to terminators
    verbose=False, # just assert
    Lmax=14)) == 4000 # all toolboxes we designed may be used simultaneously
                      # without introducing any repeat longer than 14-bp

Notice that we didn't specify `chained_bkg` as the `background` in the `Finder` job above, because it already contains $15$-mers from the previous four toolboxes, and as such would flag all parts from these toolboxes inside `all_toolboxes` as being repetitive with respect to the `background`. We will now dispense off with `chained_bkg` since it has served its purpose, and conclude this section.

In [105]:
chained_bkg.drop() # goodbye!

True

### And Now, Our Watch is Ended

We hope this notebook is useful to you in learning how to use the `Non-Repetitive Parts Calculator` effectively. We hope to convince you that the `Non-Repetitive Parts Calculator` can be a useful tool in your arsenal in your quest for genetic systems engineering. We had a lot of fun developing this notebook, and we hope you'll share it with your students and colleagues who might benefit from the `Non-Repetitive Parts Calculator`. Despite our thrust on clarity, if any part of this notebook remains ambiguous or unclear to you, please reach the authors, who'd be more than delighted to explain or update this notebook accordingly.

We'd like to stress that the genetic parts discussed above are not the only ones that can be designed using the `Non-Repetitive Parts Calculator`. This algorithm and notebook is left to our synthetic biology colleagues everywhere to help them engineer ever-larger and stable genetic systems.

### References

* Reis, A. C., Halper, S. M., Vezeau, G. E., Cetnar, D. P., Hossain, A., Clauer, P. R., and Salis, H. M. (2019). Simultaneous repression of multiple bacterial genes using nonrepetitive extra-long sgRNA arrays. Nature Biotechnology, 37(11), 1294-1301.


* Larson, M. H., Gilbert, L. A., Wang, X., Lim, W. A., Weissman, J. S., and Qi, L. S. (2013). CRISPR interference (CRISPRi) for sequence-specific control of gene expression. Nature Protocols, 8(11), 2180-2196.


* Salis, H. M., Mirsky, E. A., and Voigt, C. A. (2009). Automated design of synthetic ribosome binding sites to control protein expression. Nature Biotechnology, 27(10), 946-950.


* Green, A. A., Silver, P. A., Collins, J. J., and Yin, P. (2014). Toehold switches: de-novo-designed regulators of gene expression. Cell, 159(4), 925-939.


* Chen, Y. J., Liu, P., Nielsen, A. A., Brophy, J. A., Clancy, K., Peterson, T., and Voigt, C. A. (2013). Characterization of 582 natural and synthetic terminators and quantification of their design constraints. Nature Methods, 10(7), 659-664.


* Nielsen, A. A., Der, B. S., Shin, J., Vaidyanathan, P., Paralanov, V., Strychalski, E. A., Ross, D., Densmore, D., and Voigt, C. A. (2016). Genetic circuit design automation. Science, 352(6281).


* Hossain, A., Lopez, E., Halper, S. M., Cetnar, D. P., Reis, A. C., Strickland, D., Klavins, E. and Salis, H. M. (2020). Automated design of thousands of nonrepetitive parts for engineering stable genetic systems. Nature Biotechnology, 1-10.